In [1]:
import cv2  # We're using OpenCV to read video, to install !pip install opencv-python
import base64
import time
from openai import OpenAI
import os
import requests
# from playsound import playsound
from IPython.display import display, Image, Audio
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
client = OpenAI()

video = cv2.VideoCapture("Video1.mp4")

base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

video.release()
print(len(base64Frames), "frames read.")

913 frames read.


In [7]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames from a video. Tell me what is happening in the video as you are describing it to a blind logical person. Do not write anything unnecessary and that is a strict warning!",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::int(len(base64Frames) / 25)]),
        ],
    },
]
params = {
    "model": "gpt-4-vision-preview",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 500,
}

result = client.chat.completions.create(**params)
print(result.choices[0].message.content)

The video shows a sequence of events alongside a road with a grassy, sloping terrain to one side. Initially, two bison appear: one is lying on the grass by the roadside, and the other stands in the adjacent lane, causing vehicles to slow down or stop. The bison on the road then walks along the shoulder, followed closely behind by a second bison that rises up from its resting position. Together, they move steadily across the frame, parallel to the road. The bison navigate the edge of the road, veering slightly into the driving lane at times, which prompts motorists to pause and give them space. A vehicle's perspective is shown, pausing for the bison to walk directly in front of it, crossing to the other side of the road. The driver inside the vehicle appears to be taking photos or recording the bison as it moves by. Eventually, the bison continue off the road and the vehicle drives past them, resuming the journey as the bison settle back into the grassy area away from the traffic. The e

In [8]:
response = requests.post(
    "https://api.openai.com/v1/audio/speech",
    headers={
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    },
    json={
        "model": "tts-1-1106",
        "input": result.choices[0].message.content,
        "voice": "onyx",
    },
)

audio = b""
for chunk in response.iter_content(chunk_size=1024 * 1024):
    audio += chunk

with open('test1.wav', 'wb') as f:
    f.write(audio)